## Streamlining Walmart's Sales Data Analysis with Polars and Pandas

Data analysis is an essential aspect of retail operations, especially for a company like Walmart with vast amounts of sales data. Managing, cleaning, and visualizing this data efficiently can provide invaluable insights. In this article, we'll explore how to streamline the process of handling Walmart's sales data using Python libraries such as Pandas, Polars, and Seaborn.

We'll begin by importing the necessary libraries and loading the datasets. Polars, a high-performance DataFrame library, will be our main tool for data manipulation, while Pandas will be used for specific tasks where it excels. Seaborn and Matplotlib will help us create insightful visualizations.

First, let's set up our environment and load the data:


In [12]:
# Import libraries
import pandas as pd
import polars as pl
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from datetime import date

# Define file paths
path_sales = '/Users/fredrik.hornell/Python/Private/Walmart_M5/data/raw/sales_train_evaluation.parquet'
path_calendar = '/Users/fredrik.hornell/Python/Private/Walmart_M5/data/raw/calendar.parquet'
path_price = '/Users/fredrik.hornell/Python/Private/Walmart_M5/data/raw/sell_prices.parquet'

# Load datasets
df_sale = pl.read_parquet(path_sales)
df_calendar = pl.read_parquet(path_calendar)
df_prices = pl.read_parquet(path_price)

# Display basic information
print(df_sale.shape)
print(df_sale.sample(5))

(30490, 1947)
shape: (5, 1_947)
┌────────────────┬───────────────┬─────────────┬───────────┬───┬────────┬────────┬────────┬────────┐
│ id             ┆ item_id       ┆ dept_id     ┆ cat_id    ┆ … ┆ d_1938 ┆ d_1939 ┆ d_1940 ┆ d_1941 │
│ ---            ┆ ---           ┆ ---         ┆ ---       ┆   ┆ ---    ┆ ---    ┆ ---    ┆ ---    │
│ str            ┆ str           ┆ str         ┆ str       ┆   ┆ i64    ┆ i64    ┆ i64    ┆ i64    │
╞════════════════╪═══════════════╪═════════════╪═══════════╪═══╪════════╪════════╪════════╪════════╡
│ HOUSEHOLD_2_32 ┆ HOUSEHOLD_2_3 ┆ HOUSEHOLD_2 ┆ HOUSEHOLD ┆ … ┆ 0      ┆ 0      ┆ 0      ┆ 0      │
│ 6_CA_4_evaluat ┆ 26            ┆             ┆           ┆   ┆        ┆        ┆        ┆        │
│ io…            ┆               ┆             ┆           ┆   ┆        ┆        ┆        ┆        │
│ FOODS_3_793_CA ┆ FOODS_3_793   ┆ FOODS_3     ┆ FOODS     ┆ … ┆ 3      ┆ 3      ┆ 4      ┆ 2      │
│ _2_evaluation  ┆               ┆             ┆           

### Exploring the Data
After loading the datasets, we need to understand the structure and contents of our sales data. This includes examining the various identifiers such as item_id, dept_id, cat_id, store_id, and state_id. Sampling these columns helps us get a sense of the data.

In [13]:
print(df_sale.select("id").sample(3))
print(df_sale.select("item_id").sample(3))
print(df_sale.select("dept_id").sample(3))
print(df_sale.select("cat_id").sample(3))
print(df_sale.select("store_id").sample(3))
print(df_sale.select("state_id").sample(3))

shape: (3, 1)
┌─────────────────────────────────┐
│ id                              │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ HOUSEHOLD_2_301_CA_4_evaluatio… │
│ HOBBIES_2_066_TX_2_evaluation   │
│ HOUSEHOLD_1_457_WI_2_evaluatio… │
└─────────────────────────────────┘
shape: (3, 1)
┌─────────────────┐
│ item_id         │
│ ---             │
│ str             │
╞═════════════════╡
│ HOBBIES_2_112   │
│ FOODS_3_118     │
│ HOUSEHOLD_1_022 │
└─────────────────┘
shape: (3, 1)
┌─────────────┐
│ dept_id     │
│ ---         │
│ str         │
╞═════════════╡
│ HOUSEHOLD_1 │
│ HOUSEHOLD_2 │
│ HOBBIES_1   │
└─────────────┘
shape: (3, 1)
┌───────────┐
│ cat_id    │
│ ---       │
│ str       │
╞═══════════╡
│ FOODS     │
│ HOBBIES   │
│ HOUSEHOLD │
└───────────┘
shape: (3, 1)
┌──────────┐
│ store_id │
│ ---      │
│ str      │
╞══════════╡
│ WI_3     │
│ WI_3     │
│ WI_1     │
└──────────┘
shape: (3, 1)
┌──────────┐
│ state_id │

### Transforming the Data
To analyze daily sales trends and product performance more effectively, we need to transform the dataset into a more suitable format. This transformation involves two main steps: creating a "sales" DataFrame for sales data and a "product-location" DataFrame for metadata. This division helps in streamlining the analysis process and making the data more manageable from a memory perspectiva, have in mind this is done on a lapttop with 8 gb ram.

#### Creating the Sales DataFrame
We start by creating a "sales" DataFrame that focuses on daily sales data. This step involves melting the original sales DataFrame to convert it from a wide format (with days as columns) to a long format (with days as rows), and joining it with the calendar data to include the date information.

In [15]:
df_sales = (
    df_sale.lazy()
    .drop(['item_id','dept_id','cat_id','store_id','state_id'])
    .melt(id_vars='id')
    .rename({'variable': 'day', 'value': 'sales'})
    .with_columns(pl.col('sales').cast(pl.Int16))
    .join(df_calendar.select([pl.col("date"), pl.col("d").alias("day")]).lazy(), on="day", how="inner")
    .drop(['day'])
    .select(pl.col('id'), pl.col('date'), pl.col('sales'))
    .collect()
)
print(f"df_sale has shape {df_sale.shape}\ndf_sales has shape {df_sales.shape}")

df_sale has shape (30490, 1947)
df_sales has shape (59181090, 3)


**Why do we do this?**

1. **Simplifying Time Series Analysis:** By converting the data into a long format, each row represents a unique product-location combination for a specific day. This format simplifies the analysis of sales trends over time, making it easier to perform operations like aggregations, filtering, and time series analysis.

2. **Including Date Information:** Joining with the calendar data allows us to include the actual date information in the DataFrame. This is essential for any time-based analysis, such as identifying trends, seasonality, and other temporal patterns.

3. **Memory Efficiency:** The transformation to a long format significantly reduces memory usage when performing operations. Wide DataFrames with many columns can be less efficient in terms of memory, especially when dealing with missing or sparse data. A long format is more memory-efficient for large datasets, as it reduces redundancy and optimizes storage.

### Creating the Product-Location Metadata DataFrame
Next, we generate a "product-location" DataFrame that combines product and location metadata with sales data. This DataFrame includes important metadata such as item ID, department ID, category ID, store ID, and state ID. We also calculate metrics like the first and last transaction dates, total unit sales, and create a timeseries date range for each product-location combination.

In [17]:
df_prodloc = (
    df_sale.lazy()
    .select(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])
    .join(df_sales.lazy(), on="id", how="inner")
    .group_by(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])
    .agg([
        (pl.col("date").filter(pl.col("sales") > 0)).min().alias("first_tran_dt"),
        (pl.col("date").filter(pl.col("sales") > 0)).max().alias("last_tran_dt"),
        pl.col("date").max().alias("dataset_last_dt"),
        pl.col("sales").sum().alias("total_unit_sales")
    ])
    .with_columns(pl.col('first_tran_dt').map_elements(lambda x: pl.date_range(x, date(2016, 5, 22), interval="1d", eager=True)).alias('timeseries_daterange'))
    .collect()
)
print(df_prodloc.shape)


/var/folders/vg/gffx6m9j07j37nkdpql4hwr00000gq/T/ipykernel_42817/3834817002.py:13: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  .collect()


(30490, 11)


**Why do we do this?**

1. **Aggregating Metadata:** This DataFrame aggregates important metadata and sales metrics for each product-location combination. It provides a summary of each product's performance in different locations, including the first and last transaction dates and total sales.

2. **Facilitating Detailed Analysis:** By separating the metadata from the daily sales data, we can perform more detailed and targeted analysis. For instance, we can use this DataFrame to study sales patterns across different product categories, departments, and regions.

3. **Memory Efficiency:** Storing metadata separately from the sales data helps reduce memory usage. Metadata tends to be less volatile and smaller in size compared to daily transactional data. By separating these concerns, we can load and process large volumes of sales data more efficiently without overwhelming memory resources.

4. **Creating a Complete Time Series:** The timeseries_daterange column ensures that we have a complete date range for each product-location combination, even if no sales were recorded on some days. This completeness is crucial for accurate time series analysis and forecasting

### Creating the Adjusted Sales DataFrame
To ensure each product-location combination has a complete and accurate time series of sales data, we create an adjusted dataset df_sale_adj. This step involves generating a row for every date within the specified range and ensuring that periods before a product's introduction (where no sales occurred) are excluded. This helps in creating a realistic view of the product's sales lifecycle. This leads to ca 20 % reduction in rows in the sales datafram 



In [19]:
df_sales_adj = (
    df_prodloc.lazy()
    .select([pl.col('id'), pl.col('timeseries_daterange').alias('date')])
    .explode('date')  # Ensures every date within the range is included
    .join(df_sales.lazy(), on=['id', 'date'], how='inner')  # Join with sales data
    .collect()  # Collect the results into a DataFrame
)

print(df_sales.shape)
print(df_sales_adj.shape)


(59181090, 3)
(46796220, 3)


### Summary

Transforming Walmart's sales data into structured formats significantly enhances our ability to analyze and derive insights efficiently. Here's an overview of the process and its benefits:

Firstly, we restructure the original sales data into a "sales" DataFrame (*df_sales* & *df_sales_adj*). This transformation involves converting the data into a long format and incorporating crucial date information. By simplifying the data structure and optimizing memory usage, we facilitate streamline data handling.

Additionally, we create a "Product-Location Metadata" DataFrame (df_prodloc) to consolidate essential metadata and sales metrics for each product-location combination. This aggregation enables detailed analysis across product categories, departments, and regions while enhancing memory efficiency by separating metadata from transactional data.

Lastly, to ensure data integrity and efficient future access, we save these transformed DataFrames as parquet files (df_sales.parquet, df_sales_adj.parquet, df_prodloc.parquet). This approach not only supports scalable data management but also ensures that our analyses are both insightful and resource-efficient.

In summary, restructuring Walmart's sales data into these optimized formats empowers us to uncover meaningful insights.








